# Wave propagation in Time Domain

In this section we discuss how to solve the PDE for 2D sonic wave equation in time domain using `esys.escript`. It is assumed that you have worked through the [introduction section on `esys.escript`](escriptBasics.ipynb). 
First we will provide the basic theory.


## Sonic Wave Equation in Time domain

The sonic wave equation in time domain can be written as a system of first order differential equations:
\begin{equation}\label{eqWAVEF1}
\dot{\mathbf{V}} = - \nabla p 
\end{equation}
\begin{equation}\label{eqWAVEF2}
\frac{1}{c^2} \dot{p} + \nabla^t \mathbf{V} = \delta_{\mathbf{x}_s} \cdot w(t)
\end{equation}
where $p$ is the pressure (or volume change) and 
$\mathbf{V}$ it gradient. Source at location $\mathbf{x}_s$. $c$ is the wave propagation speed.
The dot refers time derivative. $\mathbf{x}_s$ is the location of the source,  
$\delta_{\mathbf{x}_s}$ is Dirac $\delta$-function and $w(t)$ the source wavelet as function of time $t$.
In more details these equations are given as  
\begin{equation}\label{eqWAVEF3}
\dot{\mathbf{V}} = -\begin{bmatrix}
\frac{\partial p}{\partial x_0 }\\
 \frac{\partial p}{\partial x_1 }
\end{bmatrix}
\end{equation}
\begin{equation}\label{eqWAVEF4}
\frac{1}{c^2} \dot{p} + \frac{\partial V_0}{\partial x_0 }+
 \frac{\partial V_1}{\partial x_1 } = \delta_{\mathbf{x}_s}  \cdot w(t)
\end{equation}
For the wavelet we use the [Ricker wavelet](http://subsurfwiki.org/wiki/Ricker_wavelet)

In [1]:
f = 15       # peak frequency of the Ricker wavelet
dt=0.001     # time resolution
tend=0.75    # end of wavelet
%matplotlib inline

We use the implementation in `esys.escript` which is a bit more suitable for us them the [scipy version](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.ricker.html):

For plotting we evaluate the Ricker over the interval 0 to `tend`:

We also want to take a look at the power spectrum. We get the Fourier coefficients and frequencies using `np.fft`.
Notice that we are using the real value fast fourier transformation (FFT) as the values of `signal` are real.
As a consequence only the spectrum for positive frequencies needs to be inspected: 

And we plot the power spectrum over frequency:

To assess the requirement is terms of grid sizes for solving the wave equation we need to have an idea of the 
largest frequency making contributions to the signal of the wavelet. To do this we first
need to collect a list of all frequencies for which the spectrum lays above a threshold. Here we
use $0.001$ times the maximum spectrum value to get an index of all frequencies that have 
significant contribution to the signal:

Then we grab the relevant frequencies from the array of frequencies and get the max:

## Domain set up

We consider a single reflector set up. The reflector is located at a depth of $500m$
where the top layer has propagation speed of $c_{top}=1500m/s$ 
and the bottom layer has propagation speed of $c_{top}=3000m/s$.
Use a frequency of $f=5 Hz$. The domain has depth of $1km$ and width $3km$ 
with $300 \times 100$ grid.
The source is located at the surface at an offset of $1500m$ from the boundary.

 <img src="SonicImage1.png" alt="Sonic domain" width="800"> 

This time we use a `sys.speckley` domain which is more suitable for wave problems. It actual uses higher 
order polynomial approximations:

In [9]:
nx=100  # elements in horizontal direction
ny=60   # elements in vertical direction
dx=15   # element size
order=6 # element order

Again we use a single source at the center of the top edge of the domain:

Velocity configuration is

In [12]:
c_top=1800      # m/s
c_bottom=3000      # m/s
d0=Depth-200.    # m top layer is 200m thick

Set up of the velocity $c$:

We solve the wave equation using the Heun scheme which is 2-nd order Runge-Kutta method with time step size: Starting from $U^{(0)}$:
\begin{equation}\label{eq:H1}
\hat{U}^{(n+1)} =  U^{(n)} + h \cdot F(U^{(n)}, t^{(n)}) 
\end{equation}
\begin{equation}\label{eq:H2}
\hat{U}^{(n+1)} = U^{(n)} + \frac{h}{2} (F(U^{(n)}, t^{(n)})+F(\hat{U}^{(n+1)}, t^{(n+1)}) )
\end{equation}
In our case:
\begin{equation}\label{eqWAVEF8}
U=\begin{bmatrix}
\mathbf{V}\\
 p
\end{bmatrix}
\end{equation}
and 
\begin{equation}\label{eqWAVEF9}
F=\begin{bmatrix}
-\nabla p\\
 dp
\end{bmatrix}
\end{equation}
with 
\begin{equation}\label{eqWAVEF10}
\frac{1}{c^2} dp = - \nabla^t \mathbf{V} + \delta_{\mathbf{x}_s} \cdot w(t)
\end{equation}
for which we use the `LinearSinglePDE` with $D=\frac{1}{c^2}$, $X=-\mathbf{V}$ and
$y_{dirac}=\delta_{\mathbf{x}_s} \cdot w(t)$

Define the PDE:

In [14]:
from esys.escript.linearPDEs import LinearSinglePDE, SolverOptions
pde = LinearSinglePDE(domain)
pde.getSolverOptions().setSolverMethod(SolverOptions.LUMPING)

Setting $D$ is the PDE:

In [15]:
pde.setValue(D=1./(c**2) )

An define the Dirac $\delta$ function for the source `source66` which we will later multiply with the wavelet value $(w(t)$ to define $y_{dirac}$: 

In [16]:
input_loc=Scalar(0., DiracDeltaFunctions(domain))
input_loc.setTaggedValue("source66", 1.)

The wave length $\lambda$ needs to smaller then the node spacing in order to be able to resolve
the wave and its gradient accurately (note $2\pi f \lambda=c$):

## Time integration

Also should the changes between to time steps not be two dramatic in the sense that the wave from should not
travel more than then a node distance within a time step. This leads to the condition $h<h_{max}$:

** Note **: if the time step is two large then solution can become unstable. The exact maximum size 
    depends on additional factors such as the time integration scheme and the spatial discretization scheme.
    In some an exact correction factor is known but here we use $h_{max}$ as a guidance to find the appropriate step size.

This is a Heun step:

We want to track the solution at three locations at the surface:

And also collect over a set of virtual geophones to create a synthetic seismic survey:

We write a function to progress the time integration from some time `t` to some 
time `tend` and collect the values at the geophones over time is lists:

In [38]:
Ts=[]
Trace0=[]
Trace1=[]
Trace2=[]
Traces=[]
h=0.00004
n=int(dt/h)
print("n= ",n)

def progress(p, v, t, tend):
    while t < tend:
        t, p, v = stepHeun(t, p, v, n, h)
        Ts.append(t)
        Trace0.append(loc0(p))
        Trace1.append(loc1(p))
        Trace2.append(loc2(p))
        Traces.append(loc(p))
        #print(Ts[-1], Trace0[-1], Trace1[-1], Trace2[-1])
    return t, p, v

n=  25


In [39]:
# start time integration
v=Vector(0., Function(domain))
p=Scalar(0., Solution(domain))
t=0    

t, p, v = progress(p, v, t, tend*0.2)

Lets plot the solution at that time:

And now progress to $40\%$ end time:

Lets plot the solution at that time:

And now progress to 60%
end time:

Lets plot the solution at that time:

And now progress to 80% end time:

## Seismograms

Plot the traces at the monitoring points: